In [51]:
import string
import numpy as np

#settings
window_size = 3
text_file = 'R52.txt'

#function
def preprocess_word(word):
    word = word.lower().strip()
    
    for punc in string.punctuation:
        word = word.replace(punc,"")
    return word

#main
#initialize dataset
word_training_pairs = [] #pair is (target, context)

#load text
all_sentences = open(text_file).readlines()

#extract context i.e.
for sentence in all_sentences:
    sentence_splitted = [preprocess_word(word) for word in sentence.split()]
    for i, target in enumerate(sentence_splitted):
        #print(i, target)
        for j in range(1,4):
            if not i+j >= len(sentence_splitted):
                word_training_pairs.append((target, sentence_splitted[i + j]))
            if not i-j < 0:
                word_training_pairs.append((target, sentence_splitted[i - j]))
                
id2word = list(set([pair[0] for pair in word_training_pairs]))
word2id = {w:i for i,w in enumerate(id2word)}

word_training_pairs = [(word2id[pair[0]],word2id[pair[1]]) for pair in word_training_pairs]

In [52]:
len(id2word)

22291

In [3]:
"""corpus = ["I have iMac","I have Macbook"]
corpus = [s.split() for s in corpus]

word2id = {"I":0, "have":1, "iMac":2, "Macbook":3}"""
"""[[word2id[w] for w in s] for s in corpus]"""

'[[word2id[w] for w in s] for s in corpus]'

## word2vec code

In [53]:
import tensorflow as tf
import numpy as np
import sys

In [54]:
tf.reset_default_graph()
vocabulary_size = len(id2word)
embedding_size = 50
batch_size = 1024

#Variables
w_embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size]))
softmax_weights = tf.Variable(tf.random_uniform([embedding_size,vocabulary_size]))

#Input
train_pairs = tf.placeholder(tf.int32,shape = [None, 2])
train_input = train_pairs[:,0]
train_output = train_pairs[:,1]

#Model
w_embd = tf.nn.embedding_lookup(w_embeddings, train_input)
prediction = tf.matmul(w_embd, softmax_weights)

#loss
loss = tf.losses.sparse_softmax_cross_entropy(train_output, prediction)

#optimizer
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

init = tf.global_variables_initializer()

loss_hist = []
iter = 0
with tf.Session() as sess:
    sess.run(init)
    for i in range(0,len(word_training_pairs),batch_size):
        batch_data = word_training_pairs[i:i+batch_size]
        _, loss_value = sess.run([optimizer, loss], feed_dict = {train_pairs:batch_data})
        loss_hist.append(loss_hist)
        
        sys.stdout.write("\r%d %f"%(iter,loss_value))
        sys.stdout.flush()
        iter+=1
        
    W=sess.run(w_embeddings)

4260 7.8348060

In [61]:
np.save("W.npy", W)

In [62]:
f = open("words.txt", "w")
for w in id2word:
    f.write(w+"\n")
f.close()

## 3

In [63]:
import numpy as np

W = np.load("W.npy")
id2word = [w.strip() for w in open("words.txt")]
word2id = {w:i for i,w in enumerate(id2word)}

In [64]:
from sklearn.preprocessing import normalize
W = normalize(W)

In [65]:
def print_similar_words(word,topk=10):
    word_vec1 = W[word2id[word]]
    
    sim = np.dot(W,word_vec1)
    sim_idx = np.argsort(sim)[::-1]
    
    for idx in sim_idx[1:topk+1]:
        print(id2word[idx],sim[idx])

In [69]:
print_similar_words("street")

wall 0.561401
department 0.467748
sought 0.466491
people 0.42628
stocks 0.405273
firms 0.403858
loral 0.396504
economics 0.396348
comment 0.391773
clayton 0.386121
